<a href="https://colab.research.google.com/github/mattfredericksen/CSCE-4205-ML-Project/blob/main/feature_extraction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Setup

In [42]:
import os
import json
import gzip
from urllib.request import urlopen
from contextlib import suppress

import pandas as pd
from sklearn.model_selection import train_test_split

**Dataset links**
- [Books (\~30 million, too large!)](http://deepyeti.ucsd.edu/jianmo/amazon/categoryFilesSmall/Books_5.json.gz)  
- [Clothing, Shoes, and Jewelry (\~11 million)](http://deepyeti.ucsd.edu/jianmo/amazon/categoryFilesSmall/Clothing_Shoes_and_Jewelry_5.json.gz)    
- [Electronics (\~7 million)](http://deepyeti.ucsd.edu/jianmo/amazon/categoryFilesSmall/Electronics_5.json.gz  )  
- [Home and Kitchen (\~7 million)](http://deepyeti.ucsd.edu/jianmo/amazon/categoryFilesSmall/Home_and_Kitchen_5.json.gz)  
- [Movies and TV (\~3.5 million)](http://deepyeti.ucsd.edu/jianmo/amazon/categoryFilesSmall/Movies_and_TV_5.json.gz)  

In [43]:
!wget http://deepyeti.ucsd.edu/jianmo/amazon/categoryFilesSmall/Movies_and_TV_5.json.gz

--2020-11-04 18:36:48--  http://deepyeti.ucsd.edu/jianmo/amazon/categoryFilesSmall/Movies_and_TV_5.json.gz
Resolving deepyeti.ucsd.edu (deepyeti.ucsd.edu)... 169.228.63.50
Connecting to deepyeti.ucsd.edu (deepyeti.ucsd.edu)|169.228.63.50|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 791322468 (755M) [application/octet-stream]
Saving to: ‘Movies_and_TV_5.json.gz.1’

Movies_and_TV_5.jso 100%[===================>] 754.66M  51.4MB/s    in 15s     

2020-11-04 18:37:03 (50.0 MB/s) - ‘Movies_and_TV_5.json.gz.1’ saved [791322468/791322468]



# Feature Engineering
**List of features**  
`reviewerID` - ID of the reviewer, e.g. A2SUAM1J3GNN3B  
`asin` - ID of the product, e.g. 0000013714  
`reviewerName` - name of the reviewer  
`vote` - helpful votes of the review  
`style` - a disctionary of the product metadata, e.g., "Format" is "Hardcover"  
`reviewText` - text of the review  
`overall` - rating of the product  
`summary` - summary of the review  
`unixReviewTime` - time of the review (unix time)  
`reviewTime` - time of the review (raw)  
`image` - images that users post after they have received the product  

In [44]:
def drop_features(d):
  kept_features = ("overall", "reviewText")
  return {f: d[f] for f in kept_features}

data = []

with gzip.open("Movies_and_TV_5.json.gz") as file:
  for line in file:
    with suppress(KeyError):
      data.append(drop_features(json.loads(line.strip())))

print(f'{len(data)} reviews loaded.')

3408438 reviews loaded.


In [45]:
review_data = pd.DataFrame.from_dict(data)
# del data
review_data

,overall,reviewText
0,5.0,So sorry I didn't purchase this years ago when...
1,5.0,Believe me when I tell you that you will recei...
2,5.0,"I have seen X live many times, both in the ear..."
3,5.0,"I was so excited for this! Finally, a live co..."
4,5.0,X is one of the best punk bands ever. I don't ...
...,...,...
3408433,4.0,The singing parts are very good as expected fr...
3408434,5.0,This recording of the 2015 production by the M...
3408435,4.0,I do not wish to write a review about this rel...
3408436,5.0,It was a gift.


# Training/Testing Split
In the next cell, we randomly split the data into training and testing sets. At a later time, we may want switch to using [stratified K-fold cross validation](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.StratifiedKFold.html#sklearn.model_selection.StratifiedKFold), which performs cross validation while ensuring an equal distribution of classes (star ratings).

In [46]:
targets = review_data[['overall']]
features = review_data[['reviewText']]
train_features, test_features, train_targets, test_targets = train_test_split(features, targets, test_size=0.2)